# Posterior predictive distributions for text stimuli in the experiment


$$
\newcommand{\Prob}[1]{\mathrm{P}( #1 )}
\newcommand{\given}{\vert}
\newcommand{\text}{\mathrm{text}}
\newcommand{\xtext}{w_1, w_2 \ldots w_n}
$$    

The following calculates the posterior prediction over words conditioned a text.

Given a text $\text$, the posterior predictive distribution is, informally speaking, the distribution over words that are consistent with the discourse topics of the $\text$. It is calculated as follows:
$$
\begin{align}
\mathrm{P}(w \given \phi, \text, a, m) &= \int \mathrm{P}(w \given \phi, \pi) \mathrm{P}(\pi \given \text, \phi, a, m) d\pi,\\
&= \int \big[ \sum_{\{x\}} \mathrm{P}(w \given \phi, x)\mathrm{P}(x \given \pi) \big] \mathrm{P}(\pi \given \text, \phi, a, m) d\pi
\end{align}
$$
where $\mathrm{P}(\pi \given \text, \phi, a, m)$ is the posterior distribution over topic distributions of text $\text$, with $\phi$, $a$, $m$ being the parameters of Topic model ($\phi$ is the set of $K$ component topics and $a$, $m$ are the hyper-parameters of the Dirichlet prior over the per document mixing distribution).

Note that we need to infer the posterior $\Prob{\pi \given \text, \phi, a, m}$ by sampling using a Gibbs sampler. Specifically, we iteratively sample as follows:
$$
\begin{aligned}
x_{1:n} &\sim \Prob{x_{1:n} \given w_{1:n}, \phi, \pi},\\
\pi &\sim \Prob{\pi \given x_{1:n}, a, m}.
\end{aligned}
$$
The probability distributions in both of these steps can be calculated exactly.  

## Preface
The simulations here require the use of the ipcluster, e.g. `ipcluster start -n 16`, and will take about 12 hours on a 16 core processor. The simuluations have been started with known random seeds, and so all the results will always be identical. If you set `use_cached_result` to `True` on the two instances below, the simulation results will be loaded from the cache and the simulations will not be re-run. Using this, you can get the print-out of the main results without taking any real time, i.e. in around 30 seconds.

In [1]:
from __future__ import division
import numpy
import os
import re
import configobj
import datetime
import cPickle as pickle 
from itertools import cycle

import pandas

from gustav import models

from utils import topicmodels, utils, datautils

## Check required files

* In this notebook, we do the posterior predictions based on the Topic model 2290, which has 1500 topics. We use the Gibbs sampler iterations 19000 to 20000, thinning to every 10 steps. The relevant files are stored in the `cache_directory` and the filenames are in `hdptm_samples.cfg`. 
* We will also need the `Brismo.cfg` file, which are the settings for the online *Brisbane* experiment and so contain all the texts that were used in that experiment.
* We also need the BNC corpus that was used to train Topic model 2290. This is `bnc_texts_78639361_183975_251_499.npz`.

In [2]:
cache_directory = '../cache'
cache_fullpath = lambda path: os.path.join(cache_directory, path)

filenames = {
    'experiment_cfg' : [('Brismo.cfg',
                         '909d9f8de483c4547f26fb4c34b91e12908ab5c144e065dc0fe6c1504b1f22c9')],
    'corpus' : [('bnc_texts_78639361_183975_251_499.npz',
                 '976d2ba53ecbacd092df21c4c04adf47d033fec3901e884cce69ca66ec280831')],
    'samples' : configobj.ConfigObj('hdptm_samples.cfg')\
                ['hdptm_201117172636_2290_state_checksums'].items()
}

utils.verify_cache_files(filenames['experiment_cfg'] +\
                         filenames['corpus'] +\
                         filenames['samples'],
                         cache_directory,
                         verbose=False)

Load up the corpus used by the topic models. 

In [3]:
corpus_data = numpy.load(cache_fullpath(filenames['corpus'][0][0]))
data = models.BagOfWords(**corpus_data)

Confirm which iterations of the Gibbs sampler for model 2290 we are using.

In [4]:
sample_indices = [int(x.split('_')[-1].strip('.npz')) for x, y in filenames['samples']]

Average over all the posterior samples from iteration {{min(sample_indices)}} to iteration {{max(sample_indices)}}, thinning to every 10th one. If the cached result is not used, then this will take around 10-15 mins. 

In [5]:
use_cached_result = True
cached_posterior_average_fname = 'hdptm_170617202450_2290_posterior_average.pkl'

if use_cached_result:
        
    cached_posterior_average = utils.load_pkl(cache_fullpath(cached_posterior_average_fname))
    Phi = cached_posterior_average['Phi']
    am = cached_posterior_average['am']

else:
    
    Phi = None
    am = None
    for samples_item in filenames['samples']:
        
        sample_filename, _ = samples_item 

        state = numpy.load(cache_fullpath(sample_filename))
        model = models.HierarchicalDirichletProcessTopicModel.restart(data, state, None, None)
        model.get_counts()
        phi = model.S + model.psi*model.b
        phi = (phi.T/phi.sum(1)).T

        if Phi is None:
            Phi = phi
        else:
            Phi += phi

        if am is None:
            am = model.a * model.m 
        else:
            am += model.a * model.m 
            
    n = len(filenames['samples'])
            
    Phi /= n
    am /= n

    assert numpy.allclose(Phi.sum(1), 1.0)
    utils.save_pkl(cache_fullpath(cached_posterior_average_fname), Phi = Phi, am = am)

assert utils.checksum(cache_fullpath(cached_posterior_average_fname))\
       == '0dc86b2ef9f2f53cd38925c7244612470ee16c703d4151943ecb341682c76580'

# Review inferred topics

Look at some of the inferred topics. 

In [6]:
seed = 101
K, _ = Phi.shape
print('\n'.join([models.showtopic(Phi[k], data.vocabulary, show_probability=False, K=10)
                 for k in numpy.random.RandomState(seed).permutation(K)[:50]])
     )

centre, centres, shopping, staff, community, local, facilities, run, activities, time
nature, time, flea, natural, idea, fleas, mutilation, delicate, human, passion
terms, contract, conditions, standard, parties, contracts, buyer, seller, party, business
book, published, books, guide, author, history, written, volume, edition, information
pockets, tent, bag, nylon, fabric, jacket, bags, fleece, zip, comfortable
killed, government, police, security, people, army, attack, attacks, claimed, forces
age, life, ageing, ages, time, people, youth, late, women, average
bath, water, bathroom, shower, hot, toilet, taps, cold, towel, soap
player, hamlet, pause, king, coin, heads, tragedians, time, upstage, dead
nuclear, power, reactor, fuel, plant, reactors, station, plutonium, energy, stations
mason, pageant, angel, boy, play, god, stage, wagon, father, time
carrier, bill, bills, carriers, cargo, carriage, delivery, party, freight, ocean
design, architecture, architect, architects, building, buil

In [7]:
use_cached_result = True
mcmc_sampler_seed = 20202
cached_sampler_result = 'posterior_predictions.2290.%d.pkl' % mcmc_sampler_seed
texts = topicmodels.get_experiment_texts('Brismo.cfg', cache=cache_directory)

def get_failed_to_converge(posterior_predictions, threshold=1.01):
    converged = []
    failed_to_converge = []
    for x,y in posterior_predictions:
        if numpy.round(y[1], 2) > threshold:
            failed_to_converge.append(x)
        else:
            converged.append(x)
            
    # If we use the same datasets on different cluster
    # engines, we may have a given dataset converging
    # on one engine and not another. In that case, we want to 
    # exclude that dataset from the 'failed_to_converge'
    # dataset.
    return list(set(failed_to_converge).difference(converged))

def get_arguments(failed_to_converge):
    arguments = []
    for name in cycle(failed_to_converge):
        arguments.append((name, random.randint(101, 1001)))
        if len(arguments) >= len(clients):
            break
    return arguments 
    
if not use_cached_result:
    
    from ipyparallel import Client

    model = topicmodels.PosteriorPredictive2(corpus_data, Phi, am)
    clients = Client()

    clients.block = True

    clients[:].push(dict(
        model=model,
        texts=texts)
    );

    view = clients.load_balanced_view()

    random = numpy.random.RandomState(mcmc_sampler_seed)
    arguments = zip(texts.keys(), 
                    random.randint(1001, 10001, size=len(texts.keys())))

    func = lambda argument : (argument[0],
                              model.posterior_prediction(texts[argument[0]], 
                                                         seed=argument[1], 
                                                         burn_in_iterations=100000, 
                                                         iterations=50000, 
                                                         max_attempts_to_converge=15))

    _posterior_predictions = view.map(func, arguments)
    
    posterior_predictions = {}
    for x, y in _posterior_predictions:
        posterior_predictions[x] = y[0]
    
    ###################################
    # Try again with the slow learners
    retry = 0 
    while True:
        
        failed_to_converge = get_failed_to_converge(_posterior_predictions)
        
        if len(failed_to_converge) > 0:
            
            retry += 1
            print('Retry %d: Number of failures = %d' % (retry,
                                                         len(failed_to_converge)))
            print('Failures: %s' % ','.join(failed_to_converge))
        
            arguments_retry = get_arguments(failed_to_converge)

            _posterior_predictions = view.map(func, arguments_retry)

            for x, y in _posterior_predictions:
                posterior_predictions[x] = y[0]
                
        else:
            
            break


    timestamp = datetime.datetime.now().strftime('%Y.%m.%d.%s')

    with open(cache_fullpath(cached_sampler_result), 'wb') as f:
        pickle.dump(posterior_predictions, f, protocol=2)
        
else:
    
    with open(cache_fullpath(cached_sampler_result), 'rb') as f:
        posterior_predictions = pickle.load(f)
        
assert utils.checksum(cache_fullpath(cached_sampler_result))\
   == 'e0941816a08af95379a291af9df93885fcae613e068cc6f6e051e39b78cf2742'

In [8]:
for text_name in sorted(posterior_predictions.keys(), key=lambda args: int(args.split('_')[1])):
    print(text_name)
    print('-'*len(text_name))
    print(texts[text_name])
    print('='*10)
    print(topicmodels.topic2str(posterior_predictions[text_name], corpus_data['vocabulary'], K=100))
    print('='*50)
    print('')

text_0
------
‘I don't know what I did without it’ is the sentiment. There is a difference
between fully automatic washing machines — which change the nature of the task
altogether — and ‘twin tub’ machines where the hot wet washing has to be
lifted manually into a separate drying compartment. The women who had this type
of machine complained about the considerable amount of work still required of
the housewife, and the mess on the kitchen floor to be cleared up afterwards. In
a similar way the launderette does not remove the physical drudgery of washing.
The housewife has to get the washing there in the first place, she has to unload
it, sort it, sit and watch it wash and dry (or dash out to shop in the interim)
and then pack it all up again. This, when there is a baby in the pram and a two-
or three-year-old to attend to, is no mean feat.
housework,housewife,water,washing,clean,women,time,clothes,job,wash,day,kitchen,housewives,cleaning,shop,wife,floor,satisfaction,dirty,baby,house,h

In [9]:
vocabulary = open(cache_fullpath('bnc_vocab_49324.txt')).read().split()
vocab = datautils.Vocab(vocabulary)
recalled_words = sorted(set(
    pandas.read_pickle(cache_fullpath('brisbane_06b643a_recall_results.pkl'))['word'].values)\
                        .intersection(vocabulary)
                       )

In [10]:
predictive_probabilities = []

text_names = sorted(posterior_predictions.keys(), key=lambda arg: int(arg.split('_')[1]))

for text_name in text_names:
    f = []
    for word in recalled_words:
        f.append(posterior_predictions[text_name][vocab.word2index[word]])
    predictive_probabilities.append(f)

predictive_probabilities = numpy.array(predictive_probabilities)

predictive_probabilities = numpy.c_[predictive_probabilities, 1-predictive_probabilities.sum(1)]

header = ','.join(recalled_words + ['ALTERNATIVE_WORD'])

M = [header]
for i,f in enumerate(predictive_probabilities):
    M.append(text_names[i] + ',' + ','.join(map(str, f)))
M = '\n'.join(M)

posterior_predictions_of_recalled_words = 'posterior_predictions_of_recalled_words.csv'

In [11]:
with open(cache_fullpath(posterior_predictions_of_recalled_words), 'w') as f:
    f.write(M)

# Verify the integrity of the exported csv file.
assert utils.checksum(cache_fullpath(posterior_predictions_of_recalled_words))\
    == 'd8a4155b3e8d04331ddf6c791188653b4c9612f8d395ecca286bde6983a28743'